<a href="https://colab.research.google.com/github/hyydrra/SVM-KNN-genetic/blob/master/IinteligenSystemsCA1_SVM_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

In [ ]:
train_data=pd.read_csv('/content/drive/My Drive/Fashion_MNIST/Train_Data.csv')
train_labels=pd.read_csv('/content/drive/My Drive/Fashion_MNIST/Train_Labels.csv')
test_data=pd.read_csv('/content/drive/My Drive/Fashion_MNIST/Test_Data.csv')
test_labels=pd.read_csv('/content/drive/My Drive/Fashion_MNIST/Test_Labels.csv')
dim=train_data.shape[1]
num_traindata=train_data.shape[0]
num_testdata=test_data.shape[0]
norm_train=((train_data-train_data.min())/(train_data.max()-train_data.min()))
norm_test=((test_data-test_data.min())/(test_data.max()-test_data.min()))

Question4: SVM

In [ ]:
mean, sigma = 0, 0.1 # mean and standard deviation
w0 = np.random.normal(mean, sigma, dim)

In [ ]:
def svm_sgd(X, Y,landa):
  w = w0
  eta = 1
  epochs = 50
  errors=[]
  gs=[]
  for epoch in range(1,epochs):
    #print(epoch)
    cnt=0
    for i, x in enumerate(X):
      #rmse=w**2
      #rmse=rmse.sum()
      #rmse=rmse**0.5
      #g=landa*rmse + max(0,float(1-(Y[i]*np.dot(X[i], w))))
      #gs.append(g)
      if (Y[i]*np.dot(X[i], w)) < 1:
        w = w + eta * ( (X[i] * Y[i]) + (-2  *(landa)* w) )
        cnt=cnt+1
      else:
        w = w + eta * (-2  *(landa)* w)
    eta=1/(10*epoch)
    #print("error: ",cnt)
    errors.append(cnt)
  return w,errors

In [ ]:
def svm_minibatch(X, Y,landa):
  w = w0
  eta = 1
  epochs = 50
  errors=[]
  batch=100
  for epoch in range(1,epochs):
    #print(epoch)
    #landaa=1/epoch
    landaa=landa
    cnt=0
    temp=0
    count_b=0
    for i, x in enumerate(X):
      if (Y[i]*np.dot(X[i], w)) < 1:
        temp=temp+(X[i] * Y[i])
        cnt=cnt+1
      count_b=count_b+1
      if count_b==batch:
        w = w + eta * ( (temp) + (-2  *(landaa)* w) +batch*(-2  *(landaa)* w))
        temp=0
        count_b=0
    eta=1/(10*epoch)
    #print("error: ",cnt)
    errors.append(cnt)
  return w,errors

K_fold Cross validation

In [ ]:
landas=10**np.arange(-10.0,12.0,2)
for landa in landas:
  mean=0
  for i in range(5):
    counttrue=0
    cross_test=norm_train[int(i*num_traindata/5):int((i+1)*num_traindata/5)]
    cross_testlabel=train_labels[int(i*num_traindata/5):int((i+1)*num_traindata/5)]
    Cross_train=norm_train.drop(norm_train.index[int(i*num_traindata/5):int((i+1)*num_traindata/5)])
    cross_trainlabel=train_labels.drop(train_labels.index[int(i*num_traindata/5):int((i+1)*num_traindata/5)])
    Cross_num_traindata=cross_trainlabel.shape[0]
    Cross_num_testdata=cross_testlabel.shape[0]
    Cross_weight=[]
    for n in range(10):
      curr_class=n
      trainlabel=np.zeros((Cross_num_traindata,1))
      for i in range(Cross_num_traindata):
        if int(cross_trainlabel.values[i])==curr_class:
          trainlabel[i]=1
        else:
          trainlabel[i]=-1 
      W,error=svm_sgd(Cross_train.values,trainlabel,landa)
      Cross_weight.append([])
      Cross_weight[n].append(W)
      

    for i,x in enumerate(cross_test.values):
      max=-1000000
      for j in range(10):
        temp=np.dot(cross_test.values[i],np.asarray(Cross_weight[j]).reshape(182,))
        if  temp>=max:
          max=temp
          templabel=j
      if cross_testlabel.values[i]==templabel:
        counttrue=counttrue+1
    mean=mean+counttrue
  mean=mean/5
  print("mean of accuracy for landa="+str(landa)+" is: "+str(100*mean/Cross_num_testdata))
  




RUN and plot

In [ ]:
weights=[]
for n in range(10):
  curr_class=n
  trainlabel=np.zeros((num_traindata,1))
  for i in range(num_traindata):
    if int(train_labels.values[i])==curr_class:
      trainlabel[i]=1
    else:
      trainlabel[i]=-1

  testlabel=np.zeros((num_testdata,1))
  for i in range(num_testdata):
    if int(test_labels.values[i])==curr_class:
      testlabel[i]=1
    else:
      testlabel[i]=-1

  W,error=svm_minibatch(norm_train.values,trainlabel,10**-6)
  weights.append([])
  weights[n].append(W)
  plt.plot(error)
  plt.title('Class '+str(n)+' Vs All')
  plt.xlabel('iteration')
  plt.ylabel('misclassification in train data')
  plt.show()

  counttrue=0
  for i,x in enumerate(norm_test.values):
    if  testlabel[i]*np.dot(norm_test.values[i],W)>=1:
      counttrue=counttrue+1
  print('      accuracy in test data: '+str(100*counttrue/num_testdata))


In [ ]:

counttrue=0
for i,x in enumerate(norm_test.values):
  max=-1000000
  for j in range(10):
    temp=np.dot(norm_test.values[i],np.asarray(weights[j]).reshape(182,))
    if  temp>=max:
      max=temp
      templabel=j
  if test_labels.values[i]==templabel:
    counttrue=counttrue+1

print("Total accuracy of SVM with landa=1e-6 and after 200 itteration and batch_size=100 : "+str(counttrue/i))

Total accuracy of SVM with landa=1e-6 and after 200 itteration and batch_size=100 : 0.7678673527730131


Question5: KNN

In [ ]:
#normalization
KNN_data=((train_data-train_data.min())/(train_data.max()-train_data.min()))
KNN_test=((test_data-test_data.min())/(test_data.max()-test_data.min()))

In [ ]:
#for two or more row input data
def KNN1(Input,k,FLoss):
  my_pred=[]

  for i in range(Input.shape[0]):
    count=[0,0,0,0,0,0,0,0,0,0]
    #Loss = norm2(RMSE):
    if FLoss==0:
      L=(KNN_data-Input.iloc[i,:].values.squeeze())**2
      L=L.sum(axis = 1, skipna = True)
      L=L**0.5
      
    #Loss = sum(|X-Y|):
    elif FLoss==1:
      temp=KNN_data-Input.iloc[i,:].values.squeeze()
      L=temp.abs()
      L=L.sum(axis = 1, skipna = True)

    #loss = max(|X-Y|):
    elif FLoss==2:
      temp=KNN_data-Input.iloc[i,:].values.squeeze()
      L=temp.abs()
      L=L.max(axis = 1, skipna = True)

    ind=L.nsmallest(k).index
    for j in range(k):
      for m in range(10):
        if train_labels.values[ind[j]]==m:
          count[m]=count[m]+1
    my_pred.append(count.index(max(count)))
    print(i)
  return my_pred 

In [ ]:
pred=KNN1(KNN_test,10,1)

In [ ]:
mylabels=test_labels.values
knn_error=0
for i in range(len(test_labels)):
  if int(mylabels[i])!=pred[i]:
    knn_error=knn_error+1
print((len(test_labels)-knn_error)/len(test_labels))

Qestion3: genetic

In [ ]:
charlist=list(string.ascii_lowercase)+list(string.ascii_uppercase)+[" "]
Pass="its four AM and i am still writing this lovely report"
Pass=list(Pass)
cracked=1
h=0
h1=0
H=0
cnt=0
guess=np.zeros((1,len(Pass)))
answer=[None]*len(Pass)
temp=len(Pass)
Pass=list(enumerate(Pass))
while cracked:
  guess=np.zeros((1,temp))
  for i in range(5):
    h=0
    cnt=cnt+1
    ans=np.random.randint(0,len(charlist),temp)
    for j in range(temp):
      if charlist[ans[j]]==Pass[j][1]:
        h=h+1
    if h>h1:
      h1=h
      ans1=ans
  if h1!=0:
    guess=(H*guess/(h1+H))+(h1*ans1/(h1+H))
    guess=np.round(guess)
  H=0
  h1=0
  temp_list=Pass.copy()
  for j in range(len(guess[0])):
    if charlist[int(guess[0][j])]==Pass[j][1]:
      answer[Pass[j][0]]=Pass[j][1]
      temp_list.remove(Pass[j])
      H=H+1
  Pass=temp_list.copy()
  temp=len(Pass)
  if temp==0:
    cracked=0
print(answer)
print("number of random arrays created: "+str(cnt))


['i', 't', 's', ' ', 'f', 'o', 'u', 'r', ' ', 'A', 'M', ' ', 'a', 'n', 'd', ' ', 'i', ' ', 'a', 'm', ' ', 's', 't', 'i', 'l', 'l', ' ', 'w', 'r', 'i', 't', 'i', 'n', 'g', ' ', 't', 'h', 'i', 's', ' ', 'l', 'o', 'v', 'e', 'l', 'y', ' ', 'r', 'e', 'p', 'o', 'r', 't']
number of random arrays created: 450
